In [2]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

In [3]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/17 20:04:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/17 20:04:03 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [7]:

import pandas as pd


# Sample data for Buses table
buses_data = {
    'bus_id': [1, 2, 3],
    'arrival_time': [2, 4, 7],
    'capacity': [1, 10, 2]
}

# Sample data for Passengers table
passengers_data = {
    'passenger_id': [11, 12, 13, 14, 15],
    'arrival_time': [1, 1, 5, 6, 7]
}

# # Sample data for Buses table
# buses_data = {
#     'bus_id': [1, 2, 3, 4, 5,                                    6],
#     'arrival_time': [2, 3, 7, 8, 9,                               20],
#     'capacity': [2, 1, 2, 2, 2,                                      2]
# }

# # Sample data for Passengers table
# passengers_data = {
#     'passenger_id': [1, 2, 3, 4, 5, 6],
#     'arrival_time': [1, 4, 4, 11, 12, 13]
# }

# Create DataFrames
buses_df = pd.DataFrame(buses_data)
passengers_df = pd.DataFrame(passengers_data)

# Display the DataFrames
print("Buses table:")
print(buses_df)
print("\nPassengers table:")
print(passengers_df)


df_person = spark.createDataFrame(buses_df)
df_person.createOrReplaceTempView("Buses")

df_person = spark.createDataFrame(passengers_df)
df_person.createOrReplaceTempView("Passengers")

Buses table:
   bus_id  arrival_time  capacity
0       1             2         1
1       2             4        10
2       3             7         2

Passengers table:
   passenger_id  arrival_time
0            11             1
1            12             1
2            13             5
3            14             6
4            15             7


In [8]:
query_1 = """
    select
        bus_id,
        split_id,
        row_number() over(order by bus_id, split_id) as seat_id,
        arrival_time,
        capacity
    from (
        select
            *,
            explode(sequence(1, capacity)) as split_id
        from Buses b
    ) order by bus_id, split_id
"""

In [9]:
spark.sql(query_1).show()

24/03/17 20:04:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:04:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:04:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+--------+-------+------------+--------+
|bus_id|split_id|seat_id|arrival_time|capacity|
+------+--------+-------+------------+--------+
|     1|       1|      1|           2|       1|
|     2|       1|      2|           4|      10|
|     2|       2|      3|           4|      10|
|     2|       3|      4|           4|      10|
|     2|       4|      5|           4|      10|
|     2|       5|      6|           4|      10|
|     2|       6|      7|           4|      10|
|     2|       7|      8|           4|      10|
|     2|       8|      9|           4|      10|
|     2|       9|     10|           4|      10|
|     2|      10|     11|           4|      10|
|     3|       1|     12|           7|       2|
|     3|       2|     13|           7|       2|
+------+--------+-------+------------+--------+



In [10]:
query_3 = f"""
    with 
    
    a as (
        select
            *
        from (
            select
                b.bus_id,
                b.split_id,
                b.seat_id,
                row_number() over(partition by b.bus_id, b.seat_id order by p.arrival_time, p.passenger_id asc) as order_in_split,
                p.passenger_id
            from (
                {query_1}
            ) b left join Passengers p on b.arrival_time >= p.arrival_time
            order by b.bus_id, b.split_id, p.passenger_id
        )
    ),
    
    t as (
        select
            seat_id,
            list_user_id,
            size(list_user_id) as size_of_list_user,
            count(*) over(partition by list_user_id) as size_of_group,
            row_number() over(partition by list_user_id order by seat_id) as rk_in_group
        from (
            select
                seat_id,
                collect_list(order_in_split) as list_user_id
            from a group by seat_id
        )
    ),
    
    k as (
        select
            size_of_list_user,
            size_of_group,
            least(previous_size_of_list_user, previous_size_of_group) as minus
        from (
            select
                size_of_list_user,
                size_of_group,
                lag(size_of_list_user, 1, 0) over (order by size_of_list_user asc) as previous_size_of_list_user,
                lag(size_of_group, 1, 0) over (order by size_of_list_user asc) as previous_size_of_group
            from (
                select
                    distinct
                    size_of_list_user,
                    size_of_group
                from t 
                order by size_of_list_user
            )
        )
    ),
    
    result as (
        select
            seat_id,
            list_user_id[index - 1] as user_id
        from (
            select
                *,
                row_number() over(order by seat_id) as index
            from (
                select 
                    t.*,
                    (t.size_of_list_user - k.minus) as limit_in_group 
                from t left join k on t.size_of_list_user = k.size_of_list_user
            ) where rk_in_group <= limit_in_group
        )
    )
    

"""

In [11]:
query_a = f"""
    {query_3}
    
    select * from a
"""

In [12]:
spark.sql(query_a).show(10000)

24/03/17 20:05:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+--------+-------+--------------+------------+
|bus_id|split_id|seat_id|order_in_split|passenger_id|
+------+--------+-------+--------------+------------+
|     1|       1|      1|             1|          11|
|     1|       1|      1|             2|          12|
|     2|       1|      2|             1|          11|
|     2|       1|      2|             2|          12|
|     2|       2|      3|             1|          11|
|     2|       2|      3|             2|          12|
|     2|       3|      4|             1|          11|
|     2|       3|      4|             2|          12|
|     2|       4|      5|             1|          11|
|     2|       4|      5|             2|          12|
|     2|       5|      6|             1|          11|
|     2|       5|      6|             2|          12|
|     2|       6|      7|             1|          11|
|     2|       6|      7|             2|          12|
|     2|       7|      8|             1|          11|
|     2|       7|      8|   

In [13]:
query_t = f"""
    {query_3}
    
    select * from t
"""

In [14]:
spark.sql(query_t).show()

24/03/17 20:05:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+---------------+-----------------+-------------+-----------+
|seat_id|   list_user_id|size_of_list_user|size_of_group|rk_in_group|
+-------+---------------+-----------------+-------------+-----------+
|      1|         [1, 2]|                2|           11|          1|
|      2|         [1, 2]|                2|           11|          2|
|      3|         [1, 2]|                2|           11|          3|
|      4|         [1, 2]|                2|           11|          4|
|      5|         [1, 2]|                2|           11|          5|
|      6|         [1, 2]|                2|           11|          6|
|      7|         [1, 2]|                2|           11|          7|
|      8|         [1, 2]|                2|           11|          8|
|      9|         [1, 2]|                2|           11|          9|
|     10|         [1, 2]|                2|           11|         10|
|     11|         [1, 2]|                2|           11|         11|
|     12|[1, 2, 3, 4

In [15]:
query_k = f"""
    {query_3}
    
    select * from k
"""

In [16]:
spark.sql(query_k).show()

24/03/17 20:05:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 2

+-----------------+-------------+-----+
|size_of_list_user|size_of_group|minus|
+-----------------+-------------+-----+
|                2|           11|    0|
|                5|            2|    2|
+-----------------+-------------+-----+



In [17]:
query_result = f"""
    {query_3}
    
    select * from result
"""

In [18]:
spark.sql(query_result).show()

24/03/17 20:05:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 2

+-------+-------+
|seat_id|user_id|
+-------+-------+
|      1|      1|
|      2|      2|
|     12|      3|
|     13|      4|
+-------+-------+



In [19]:
query_final_result = f"""
    {query_3}
    
    select 
        distinct
        a.bus_id,
        a1.passenger_id
    from result r left join a on r.seat_id = a.seat_id
    left join a as a1 on r.user_id = a1.order_in_split

"""

In [20]:
spark.sql(query_final_result).show(10000)

24/03/17 20:05:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 20:05:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/17 2

+------+------------+
|bus_id|passenger_id|
+------+------------+
|     1|          11|
|     2|          12|
|     3|          13|
|     3|          14|
+------+------------+

